<a href="https://colab.research.google.com/github/fengtastics/111-Capstone/blob/master/Arterra_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/

#https://www.digitalocean.com/community/tutorials/a-guide-to-time-series-forecasting-with-prophet-in-python-3

#https://towardsdatascience.com/an-end-to-end-project-on-time-series-analysis-and-forecasting-with-python-4835e6bf050b

#https://www.bounteous.com/insights/2020/09/15/forecasting-time-series-model-using-python-part-one/

In [2]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
from numpy import log
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import matplotlib

from fbprophet import Prophet

from statsmodels.tsa.arima_model import ARIMA

from statsmodels.graphics.api import qqplot


matplotlib.rcParams['axes.labelsize'] = 12
matplotlib.rcParams['xtick.labelsize'] = 10
matplotlib.rcParams['ytick.labelsize'] = 10
matplotlib.rcParams['text.color'] = 'k'

In [3]:
#raw_df = pd.read_csv("C:\\Users\\mcinn\\Documents\\timeseries_vqa.csv")
raw_df = pd.read_csv("timeseries.csv",encoding='latin-1')    
   
raw_df.head()
#raw_df.info()

,Region,AWC Fiscal Year Month Abbr,AWC FYr & Pd No,Country Group,AWC and Comp,Varietal,Brand,Channel,Unit Size,9L Cases
0,BC,F2021 DEC,202110,CANADA-VQA,COMPETITOR,KERNER,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0
1,BC,F2021 DEC,202110,CANADA-VQA,COMPETITOR,MERLOT,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0
2,BC,F2021 DEC,202110,CANADA-VQA,COMPETITOR,UNASSIGNED,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0
3,BC,F2021 DEC,202110,CANADA-VQA,COMPETITOR,ZWEIGELT,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0
4,BC,F2021 DEC,202110,CANADA-VQA,COMPETITOR,UNASSIGNED,0927363 BC LTD. - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0


In [4]:
# replace Sales Dollar , 9L Cases to NUMBERS format

#raw_df['Sales Dollars'] = raw_df['Sales Dollars'].str.replace(',', '').astype(float)
raw_df['9L Cases'] = raw_df['9L Cases'].str.replace(',', '').astype(float)
raw_df.head()



,Region,AWC Fiscal Year Month Abbr,AWC FYr & Pd No,Country Group,AWC and Comp,Varietal,Brand,Channel,Unit Size,9L Cases
0,BC,F2021 DEC,202110,CANADA-VQA,COMPETITOR,KERNER,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
1,BC,F2021 DEC,202110,CANADA-VQA,COMPETITOR,MERLOT,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
2,BC,F2021 DEC,202110,CANADA-VQA,COMPETITOR,UNASSIGNED,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
3,BC,F2021 DEC,202110,CANADA-VQA,COMPETITOR,ZWEIGELT,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
4,BC,F2021 DEC,202110,CANADA-VQA,COMPETITOR,UNASSIGNED,0927363 BC LTD. - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0


In [5]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211587 entries, 0 to 211586
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Region                      211587 non-null  object 
 1   AWC Fiscal Year Month Abbr  211587 non-null  object 
 2   AWC FYr & Pd No             211587 non-null  int64  
 3   Country Group               211587 non-null  object 
 4   AWC and Comp                211587 non-null  object 
 5   Varietal                    211567 non-null  object 
 6   Brand                       211587 non-null  object 
 7   Channel                     211587 non-null  object 
 8   Unit Size                   211587 non-null  int64  
 9   9L Cases                    211587 non-null  float64
dtypes: float64(1), int64(2), object(7)
memory usage: 16.1+ MB


In [6]:
raw_df = raw_df.rename({'AWC Fiscal Year Month Abbr': 'Date','9L Cases':'Cases'}, axis=1) 

raw_df['Date'] = raw_df['Date'].str[1:]
raw_df.head()

,Region,Date,AWC FYr & Pd No,Country Group,AWC and Comp,Varietal,Brand,Channel,Unit Size,Cases
0,BC,2021 DEC,202110,CANADA-VQA,COMPETITOR,KERNER,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
1,BC,2021 DEC,202110,CANADA-VQA,COMPETITOR,MERLOT,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
2,BC,2021 DEC,202110,CANADA-VQA,COMPETITOR,UNASSIGNED,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
3,BC,2021 DEC,202110,CANADA-VQA,COMPETITOR,ZWEIGELT,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
4,BC,2021 DEC,202110,CANADA-VQA,COMPETITOR,UNASSIGNED,0927363 BC LTD. - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0


In [7]:
import datetime
raw_df['Date'] = pd.to_datetime(raw_df['Date'])
raw_df.head()

,Region,Date,AWC FYr & Pd No,Country Group,AWC and Comp,Varietal,Brand,Channel,Unit Size,Cases
0,BC,2021-12-01,202110,CANADA-VQA,COMPETITOR,KERNER,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
1,BC,2021-12-01,202110,CANADA-VQA,COMPETITOR,MERLOT,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
2,BC,2021-12-01,202110,CANADA-VQA,COMPETITOR,UNASSIGNED,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
3,BC,2021-12-01,202110,CANADA-VQA,COMPETITOR,ZWEIGELT,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
4,BC,2021-12-01,202110,CANADA-VQA,COMPETITOR,UNASSIGNED,0927363 BC LTD. - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0


In [8]:
raw_df['Date'].min(), raw_df['Date'].max()

(Timestamp('2017-01-01 00:00:00'), Timestamp('2021-12-01 00:00:00'))

In [9]:
raw_df['Country Group'].unique()


array(['CANADA-VQA', 'CANADA-ICB'], dtype=object)

In [10]:
raw_df['Channel'].unique()

array(['IND RETAILERS - SHIPMENTS', 'BC LIQUOR STORES - OVER THE COUNTER',
       'Regular OTC Sales', 'Grocery Scan'], dtype=object)

In [11]:

#ICB
icb_df = raw_df.loc[raw_df['Country Group'] == 'CANADA-ICB']
bc_icb_df = icb_df.loc[icb_df['Region'] == 'BC']
ont_icb_df=icb_df.loc[icb_df['Region'] == 'ONT']

#VQA
vqa_df = raw_df.loc[raw_df['Country Group'] == 'CANADA-VQA']
bc_vqa_df = vqa_df.loc[vqa_df['Region'] == 'BC']
ont_vaq_df=vqa_df.loc[vqa_df['Region'] == 'ONT']


#Channel

BC_ind_df = raw_df.loc[raw_df['Channel'] == 'IND RETAILERS - SHIPMENTS']
BC_OTC_df = raw_df.loc[raw_df['Channel'] == 'BC LIQUOR STORES - OVER THE COUNTER']

ONT_groc_df = raw_df.loc[raw_df['Channel'] == 'Grocery Scan']
ONT_OTC_df = raw_df.loc[raw_df['Channel'] == 'Regular OTC Sales']



In [12]:
#audit
bc_vqa_df.head()

,Region,Date,AWC FYr & Pd No,Country Group,AWC and Comp,Varietal,Brand,Channel,Unit Size,Cases
0,BC,2021-12-01,202110,CANADA-VQA,COMPETITOR,KERNER,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
1,BC,2021-12-01,202110,CANADA-VQA,COMPETITOR,MERLOT,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
2,BC,2021-12-01,202110,CANADA-VQA,COMPETITOR,UNASSIGNED,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
3,BC,2021-12-01,202110,CANADA-VQA,COMPETITOR,ZWEIGELT,0918130 B.C. LIMITED - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0
4,BC,2021-12-01,202110,CANADA-VQA,COMPETITOR,UNASSIGNED,0927363 BC LTD. - UNASSIGNED BRAND,IND RETAILERS - SHIPMENTS,750,0.0


### TRY 1 Country group


In [13]:
bc_icb_df.head()


,Region,Date,AWC FYr & Pd No,Country Group,AWC and Comp,Varietal,Brand,Channel,Unit Size,Cases
56,BC,2021-12-01,202110,CANADA-ICB,COMPETITOR,ROSE,40 KNOTS,IND RETAILERS - SHIPMENTS,1500,0.0
58,BC,2021-12-01,202110,CANADA-ICB,COMPETITOR,GAMAY,40 KNOTS,IND RETAILERS - SHIPMENTS,750,7.0
59,BC,2021-12-01,202110,CANADA-ICB,COMPETITOR,MERITAGE,40 KNOTS,IND RETAILERS - SHIPMENTS,750,1.0
60,BC,2021-12-01,202110,CANADA-ICB,COMPETITOR,MERLOT,40 KNOTS,IND RETAILERS - SHIPMENTS,750,0.0
62,BC,2021-12-01,202110,CANADA-ICB,COMPETITOR,PINOT GRIGIO,40 KNOTS,IND RETAILERS - SHIPMENTS,750,12.0


In [14]:
bc_icb_df['Region'].unique()

array(['BC'], dtype=object)

In [15]:
bc_icb_df.isnull().sum()


Region             0
Date               0
AWC FYr & Pd No    0
Country Group      0
AWC and Comp       0
Varietal           0
Brand              0
Channel            0
Unit Size          0
Cases              0
dtype: int64

In [16]:

bc_icb_region_cases=bc_icb_df.groupby(['Date'])['Cases'].sum().reset_index()
bc_icb_region_cases.tail()

,Date,Cases
42,2021-08-01,246490.0
43,2021-09-01,185493.0
44,2021-10-01,219732.0
45,2021-11-01,212922.0
46,2021-12-01,288252.0


##  ARMA MODEL BEGIN HERE 


In [17]:
##  ARMA MODEL BEGIN HERE 

In [18]:
# VQA statistics

result = adfuller(bc_icb_region_cases['Cases'])

print('ADF Statistic: {}'.format(result[0]))
print('p-value: {}'.format(result[1]))
print('Critical Values:')
for key, value in result[4].items():
    print('\t{}: {}'.format(key, value))

NameError: ignored

In [ ]:
#
#ICB

plt.rcParams.update({'figure.figsize':(8,5), 'figure.dpi':120})
# Original Series
fig, axes = plt.subplots(3, 2, sharex=True)
axes[0, 0].plot(bc_icb_region_cases.Cases); axes[0, 0].set_title('Original Series')
sm.graphics.tsa.plot_acf(bc_icb_region_cases.Cases, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(bc_icb_region_cases.Cases.diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(bc_icb_region_cases.Cases.diff().dropna(), ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(bc_icb_region_cases.Cases.diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(bc_icb_region_cases.Cases.diff().diff().dropna(), ax=axes[2, 1])

## we need *2nd* order differencing


plt.show()

ARIMA MODEL BEGINS HERE

In [ ]:
# bc_icb_region_cases


# 0,0,0 ARIMA Model
bc_icb_model = ARIMA(bc_icb_region_cases.Cases,order=(0,0,0))
bc_icb_model_fit = bc_icb_model.fit(disp=0)
print(bc_icb_model_fit.summary())

In [ ]:
# Plot residual errors


residuals = pd.DataFrame(bc_icb_model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
# TEST and TRAIN

#https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/
# Create Training and Test
#train = bc_icb_region_cases.Cases[:85]
#test = bc_icb_region_cases.Cases[85:]

# Build Model
# model = ARIMA(train, order=(3,2,1))  
#model = ARIMA(train, order=(1, 1, 1))  
#fitted = model.fit(disp=-1)  

# Forecast
#fc, se, conf = fitted.forecast(15, alpha=0.05)  # 95% conf

# Make as pandas series
#fc_series = pd.Series(fc, index=test.index)
#lower_series = pd.Series(conf[:, 0], index=test.index)
#upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
#plt.figure(figsize=(12,5), dpi=100)
#plt.plot(train, label='training')
#plt.plot(test, label='actual')
#plt.plot(fc_series, label='forecast')
#plt.fill_between(lower_series.index, lower_series, upper_series, color='k', alpha=.15)
#plt.title('Forecast vs Actuals')
#plt.legend(loc='upper left', fontsize=8)
#plt.show()



In [ ]:
#https://www.geeksforgeeks.org/python-arima-model-for-time-series-forecasting/


# Split data into train / test sets 
train = bc_icb_region_cases.iloc[:len(bc_icb_region_cases)-12] 
test = bc_icb_region_cases.iloc[len(bc_icb_region_cases)-12:] # set one year(12 months) for testing 

  
model = ARIMA(train['Cases'],  order = (0, 0, 0),  
) 
  
result = model.fit() 
result.summary() 



In [ ]:
A
df_icb_region_cases = icb_region_cases.rename(columns={'Date': 'ds', '9L Cases': 'y'})

# VQA
df_icb_region_cases.head()

In [ ]:
#
#ICB

plt.rcParams.update({'figure.figsize':(8,5), 'figure.dpi':120})
# Original Series
fig, axes = plt.subplots(3, 2, sharex=True)
axes[0, 0].plot(df_icb_region_cases.y); axes[0, 0].set_title('Original Series')
sm.graphics.tsa.plot_acf(df_icb_region_cases.y, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(df_icb_region_cases.y.diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(df_icb_region_cases.y.diff().dropna(), ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(df_icb_region_cases.y.diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(df_icb_region_cases.y.diff().diff().dropna(), ax=axes[2, 1])

## we need *2nd* order differencing


plt.show()

In [ ]:
# 2nd order diff
#????????????  not sure if we neeed this for Prophet  #####

#df.y=df.y.diff()

In [ ]:
## Prophet Begins Here ##

In [ ]:
# prophet defaults to 80% interval.  
# set up holidays  https://github.com/dr-prodigy/python-holidays

icb_model = Prophet(interval_width=0.80)
icb_model = icb_model.add_country_holidays(country_name='CA')
icb_model.train_holiday_names


In [ ]:
icb_model.train_holiday_names

In [ ]:

icb_model.fit(df_icb_region_cases)

In [ ]:
# Frequency  = M monthly;  MS month start !?
icb_forecast = icb_model.make_future_dataframe(periods=24, freq='MS')


#predict
icb_forecast=icb_model.predict(icb_forecast)

In [ ]:
#forecasted value
#rose_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
#vqa_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
icb_forecast.head()

In [ ]:
## CROSS VALIDATION Starts HERE ------------

In [ ]:
m = Prophet(seasonality_mode='multiplicative',
                interval_width=0.95).fit(df_icb_region_cases)

future = m.make_future_dataframe(periods=12, freq='MS')
forecast = m.predict(future)

forecast = forecast[['ds','yhat_lower', 'yhat', 'yhat_upper']]

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(m, initial='900 days', period='90 days', horizon = '180 days')


In [ ]:
df_cv.head()

In [ ]:
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head(10)

In [ ]:
# calculate MAE between expected and predicted values for december
from sklearn.metrics import mean_absolute_error, mean_squared_error
y_true = df_cv['y'].values
y_pred = df_cv['yhat'][-140:].values
mae = mean_absolute_error(y_true, y_pred)
#mse = mean_squared_error (y_true, y_pred)
print('MAE: %.3f' % mae)
#print('MAE: %.3f' % mse)

In [ ]:
#y_true
y_pred

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mape')


In [ ]:
#fb prophet
from fbprophet.plot import plot_plotly, plot_components_plotly


In [ ]:
#plt.figure(figsize=(6, 3))

m.plot(forecast, xlabel = 'Date', ylabel = 'Units Sold')
plt.title('Forecast');

In [ ]:
# decomposition

#m.plot_components(forecast)

In [ ]:
#ICB
plot_plotly(m, forecast)